# install extra library

In [1]:
!pip install transformers torch
!pip install sentencepiece

# read stored data from DataPlatform

In [2]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [3]:
dir_raw_id = 64
dir_process_id = 65

In [4]:
datanode = DataNode()

Init DataNode sucessful!


In [5]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='amarin.json'))
meta

{'owner': 'suphawadi',
 'human_size': '25.32 KB',
 'name': 'amarin.json',
 'description': 'file amarin.json',
 'directory': 64,
 'path': 'suphawadi/DSI310-12/rawData/amarin.json'}

In [6]:
data = json.load(fp)
data

[{'Head': 'ตาดีได้ ตาร้ายเสีย "รถมือสองน้ำท่วม" เช็กยังไง ไม่ให้พลาด',
  'Topic': 'คงต้องบอกว่าหลังจากนี้น่าจะมี ',
  'date': '7 ต.ค. 65',
  'tags': 'ยานยนต์'},
 {'Head': 'ระทึกกลางกรุง! รถชน 6 คันซ้อน เบื้องต้นบาดเจ็บ 4 ราย การจราจรติดหนัก',
  'Topic': 'วันที่ 8 ต.ค.65 ผู้สื่อข่าวรายงาน เวลาประมาณ 18.50 น. บริเวณหน้าแพลทินัม ประตูน้ำ ถ.เพชรบุรี\xa0ทิศทาง ขาเข้า จากแยกประตูน้ำ มุ่งหน้า แยกราชเทวี เกิดเหตุรถชนกันหลายคัน ทำให้เกิดการจราจรติดขัด\xa0เบื้องต้นมีรายงานว่าอุบัติเหตุครั้งนี้มีรถชนกันกว่า 6 คันซ้อน มีผู้ได้รับบาดเจ็บ 4 ราย และได้ส่งรักษาตัวที่รพ.คามิลเลียน อุบัติเหตุครั้งนี้มีคันแรกเป็นรถเมล์',
  'date': '8 ต.ค. 65',
  'tags': 'รถชน'},
 {'Head': 'พ่อพาลูกไปกินข้าว วูบขณะขี่จยย. รถพุ่งชนอัดกำแพงบ้านดับ',
  'Topic': 'จากกรณี ที่นายมานะ (สงวนนามสกุล) อายุ 46 ปี ช่างรับเหมาก่อสร้าง ขี่จยย.พ่วงข้างพาบุตรชาย 2 คน วัย 10 ปี และ 3 ปี ไปกินข้าว แต่ขณะที่ขี่มาถึงในหมู่บ้านแห่งหนึ่ง ม.2 ต.เนินพระ อ.เมือง จ.ระยอง ได้วูบฟุบกับแฮนด์จยย.เสียชีวิตก่อรถจยย.จะพุ่งชนต้นไม้ริมกำแพง จนร่างตกลงมาที่

# NER (NameEntitiesRegcognition)

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [8]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [9]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9993357, 'index': 4, 'word': '▁Wolfgang', 'start': 10, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9990395, 'index': 9, 'word': '▁Berlin', 'start': 33, 'end': 40}]


In [10]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [18]:
data

[{'Head': 'ตาดีได้ ตาร้ายเสีย "รถมือสองน้ำท่วม" เช็กยังไง ไม่ให้พลาด',
  'Topic': 'คงต้องบอกว่าหลังจากนี้น่าจะมี ',
  'date': '7 ต.ค. 65',
  'tags': 'ยานยนต์'},
 {'Head': 'ระทึกกลางกรุง! รถชน 6 คันซ้อน เบื้องต้นบาดเจ็บ 4 ราย การจราจรติดหนัก',
  'Topic': 'วันที่ 8 ต.ค.65 ผู้สื่อข่าวรายงาน เวลาประมาณ 18.50 น. บริเวณหน้าแพลทินัม ประตูน้ำ ถ.เพชรบุรี\xa0ทิศทาง ขาเข้า จากแยกประตูน้ำ มุ่งหน้า แยกราชเทวี เกิดเหตุรถชนกันหลายคัน ทำให้เกิดการจราจรติดขัด\xa0เบื้องต้นมีรายงานว่าอุบัติเหตุครั้งนี้มีรถชนกันกว่า 6 คันซ้อน มีผู้ได้รับบาดเจ็บ 4 ราย และได้ส่งรักษาตัวที่รพ.คามิลเลียน อุบัติเหตุครั้งนี้มีคันแรกเป็นรถเมล์',
  'date': '8 ต.ค. 65',
  'tags': 'รถชน'},
 {'Head': 'พ่อพาลูกไปกินข้าว วูบขณะขี่จยย. รถพุ่งชนอัดกำแพงบ้านดับ',
  'Topic': 'จากกรณี ที่นายมานะ (สงวนนามสกุล) อายุ 46 ปี ช่างรับเหมาก่อสร้าง ขี่จยย.พ่วงข้างพาบุตรชาย 2 คน วัย 10 ปี และ 3 ปี ไปกินข้าว แต่ขณะที่ขี่มาถึงในหมู่บ้านแห่งหนึ่ง ม.2 ต.เนินพระ อ.เมือง จ.ระยอง ได้วูบฟุบกับแฮนด์จยย.เสียชีวิตก่อรถจยย.จะพุ่งชนต้นไม้ริมกำแพง จนร่างตกลงมาที่

In [19]:
output

[{'entity': 'B-ORG',
  'score': 0.698593,
  'index': 11,
  'word': 'บี',
  'start': 30,
  'end': 32},
 {'entity': 'B-ORG',
  'score': 0.87744075,
  'index': 21,
  'word': 'วิศวกรรม',
  'start': 61,
  'end': 69},
 {'entity': 'I-ORG',
  'score': 0.48162195,
  'index': 22,
  'word': 'ศาสตร์',
  'start': 69,
  'end': 75},
 {'entity': 'I-ORG',
  'score': 0.81812245,
  'index': 24,
  'word': 'นิยม',
  'start': 82,
  'end': 86},
 {'entity': 'B-ORG',
  'score': 0.99959797,
  'index': 25,
  'word': '▁ม',
  'start': 86,
  'end': 88},
 {'entity': 'I-ORG',
  'score': 0.99905485,
  'index': 26,
  'word': '.',
  'start': 88,
  'end': 89},
 {'entity': 'I-ORG',
  'score': 0.9991003,
  'index': 27,
  'word': 'เทคโนโลยี',
  'start': 89,
  'end': 98},
 {'entity': 'I-ORG',
  'score': 0.9991115,
  'index': 28,
  'word': 'สุร',
  'start': 98,
  'end': 101},
 {'entity': 'I-ORG',
  'score': 0.9994918,
  'index': 29,
  'word': 'นา',
  'start': 101,
  'end': 103},
 {'entity': 'I-ORG',
  'score': 0.99923813,
  '

In [11]:
text = data[10]['Topic']
text

'จากกรณีหนุ่มวัย 15 ปี ซิ่งเก๋งบีเอ็มดับเบิลยูฝ่าไฟแดงชนบัณฑิตวิศวกรรมศาสตร์เกียรตินิยม ม.เทคโนโลยีสุรนารี เสียชีวิตคาที่โดยเหตุเกิดเมื่อวันที่ 1 ต.ค. 65 เจ้าหน้าที่ตำรวจ สภ.โพธิ์กลาง อำเภอเมือง จังหวัดนครราชสีมา ได้รับแจ้งเกิดอุบัติเหตุบริเวณสี่แยกไฟแดง ถนนมหาวิทยาลัยเทคโนโลยี ประตู 1 ก่อนถึงประตูทางเข้ามหาวิทยาลัย ตำบลสุรนารี อำเภอเมืองนครราชสีมาทำให้นายนายธนพล แก้วมูล หรือเต้ อายุ 24 ปี ชาว อ.ท่าตะโก จ.นครสวรรค์เสียชีวิตดังกล่าว'

In [12]:
import re

In [13]:
output = nlp(text)
output

[{'entity': 'B-ORG',
  'score': 0.698593,
  'index': 11,
  'word': 'บี',
  'start': 30,
  'end': 32},
 {'entity': 'B-ORG',
  'score': 0.87744075,
  'index': 21,
  'word': 'วิศวกรรม',
  'start': 61,
  'end': 69},
 {'entity': 'I-ORG',
  'score': 0.48162195,
  'index': 22,
  'word': 'ศาสตร์',
  'start': 69,
  'end': 75},
 {'entity': 'I-ORG',
  'score': 0.81812245,
  'index': 24,
  'word': 'นิยม',
  'start': 82,
  'end': 86},
 {'entity': 'B-ORG',
  'score': 0.99959797,
  'index': 25,
  'word': '▁ม',
  'start': 86,
  'end': 88},
 {'entity': 'I-ORG',
  'score': 0.99905485,
  'index': 26,
  'word': '.',
  'start': 88,
  'end': 89},
 {'entity': 'I-ORG',
  'score': 0.9991003,
  'index': 27,
  'word': 'เทคโนโลยี',
  'start': 89,
  'end': 98},
 {'entity': 'I-ORG',
  'score': 0.9991115,
  'index': 28,
  'word': 'สุร',
  'start': 98,
  'end': 101},
 {'entity': 'I-ORG',
  'score': 0.9994918,
  'index': 29,
  'word': 'นา',
  'start': 101,
  'end': 103},
 {'entity': 'I-ORG',
  'score': 0.99923813,
  '

In [21]:
post_process(output)

[{'type': 'B-ORG', 'text': 'บี'},
 {'type': 'I-ORG', 'text': 'วิศวกรรมศาสตร์'},
 {'type': 'I-ORG', 'text': 'ศาสตร์'},
 {'type': 'I-ORG', 'text': 'นิยม ม.เทคโนโลยีสุรนารี'},
 {'type': 'I-ORG', 'text': ' ม.เทคโนโลยีสุรนารี'},
 {'type': 'I-ORG', 'text': 'เทคโนโลยีสุรนารี'},
 {'type': 'I-ORG', 'text': 'สุรนารี'},
 {'type': 'I-ORG', 'text': 'นารี'},
 {'type': 'I-ORG', 'text': 'รี'},
 {'type': 'I-LOC', 'text': 'โพธิ์กลาง'},
 {'type': 'I-LOC', 'text': 'กลาง'},
 {'type': 'B-LOC', 'text': 'นครราชสีมา'},
 {'type': 'I-LOC', 'text': 'ไฟแดง ถนนมหาวิทยาลัยเทคโนโลยี'},
 {'type': 'I-LOC', 'text': 'แดง ถนนมหาวิทยาลัยเทคโนโลยี'},
 {'type': 'I-LOC', 'text': ' ถนนมหาวิทยาลัยเทคโนโลยี'},
 {'type': 'I-LOC', 'text': 'มหาวิทยาลัยเทคโนโลยี'},
 {'type': 'I-LOC', 'text': 'เทคโนโลยี'},
 {'type': 'I-LOC', 'text': 'ประตู'},
 {'type': 'I-LOC', 'text': ' ตําบลสุรนารี'},
 {'type': 'I-LOC', 'text': 'สุรนารี'},
 {'type': 'I-LOC', 'text': 'นารี'},
 {'type': 'I-LOC', 'text': 'รี'},
 {'type': 'B-LOC', 'text': 'นครราชสีมา'}

# Process news to spo

In [28]:
spo = []
for amarin in tqdm(data):
    Head = amarin.get('Head') 
    if Head == None:
        continue
    spo.append({
        'Head': amarin.get('Head', ""),
        'Topic': amarin.get('Topic', ""),
        'date': amarin.get('date', ""),
        'tags': amarin.get('tags', ""),
        })

  0%|          | 0/12 [00:00<?, ?it/s]

In [29]:
df = pd.DataFrame(spo)
df

,Head,Topic,date,tags
0,"ตาดีได้ ตาร้ายเสีย ""รถมือสองน้ำท่วม"" เช็กยังไง...",คงต้องบอกว่าหลังจากนี้น่าจะมี,7 ต.ค. 65,ยานยนต์
1,ระทึกกลางกรุง! รถชน 6 คันซ้อน เบื้องต้นบาดเจ็บ...,วันที่ 8 ต.ค.65 ผู้สื่อข่าวรายงาน เวลาประมาณ 1...,8 ต.ค. 65,รถชน
2,พ่อพาลูกไปกินข้าว วูบขณะขี่จยย. รถพุ่งชนอัดกำแ...,จากกรณี ที่นายมานะ (สงวนนามสกุล) อายุ 46 ปี ช่...,23 ต.ค. 65,อุบัติเหตุ
3,เปิดใจครอบครัวหนุ่มวัย 15 ขับรถชนบัณฑิต รับเสี...,วันที่ 5 ต.ค.65 กรณีเยาวชนชายวัยรุ่น อายุ 15 ป...,5 ต.ค. 65,รถชน
4,เก๋งคณะผู้แสวงบุญพร้อมพระ ยูเทิร์นไม่ทันระวัง ...,วันที่ 23 ต.ค.65 ผู้สื่อข่าวรายงานว่า ตำรวจ สภ...,23 ต.ค. 65,อุบัติเหตุ
5,ชายสติไม่ดี หยิบเหล้าเซ่นศาลยกซด ผ่านไป 4 วันพ...,เมื่อช่วงค่ำที่ผ่านมา (18 ต.ค.65) ร.ต.อ.ขัตติย...,19 ต.ค. 65,ศาลหมู่บ้าน
6,สาววูบเพราะยาแก้แพ้ ขับรถชนต้นไม้บาดเจ็บสาหัส ...,เมื่อเวลา 01.30 น วันที่ 23 ตุลาคม 2565 ร.ต.อ....,23 ต.ค. 65,อุบัติเหตุ
7,วัยรุ่นซิ่ง จยย. ไล่กวดย้อนศร คันแรกสวนกระบะไป...,เมื่อเวลา 02.30 น. วันที่ 19 ต.ค. พ.ต.ท.ธนบดี ...,19 ต.ค. 65,อุบัติเหตุ
8,ครอบครัวหนุ่ม 15 ซิ่งเก๋งชนบัณฑิต ยังไม่พร้อมใ...,จากกรณีอุบัติเหตุเมื่อคืนวันที่ 30 กันยายน 256...,3 ต.ค. 65,รถชน
9,ทนายเตรียมช่วยดูแลคดี บัณฑิตใหม่ ถูกเยาวชนซิ่ง...,จากกรณีอุบัติเหตุเมื่อคืนวันที่ 30 กันยายน 256...,2 ต.ค. 65,รถชน


In [30]:
datanode.write(df=df, directory=dir_process_id, name="spo", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='amarin.json')])

File spo.parquet alrady exists, do you want to replace y/n :  y


{'sucess': True,
 'file_id': 79,
 'path': 'suphawadi/DSI310-12/processData/spo.parquet',
 'lineage': True}